In [1]:
import torch
import torch.nn as nn
from torch.distributions import Categorical
import gym
import numpy as np 

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = "cpu"

In [2]:
class Memory:
    def __init__(self):
        self.actions = []
        self.states = []
        self.logprobs = []
        self.rewards = []
        self.is_terminals = []
    
    def clear_memory(self):
        del self.actions[:]
        del self.states[:]
        del self.logprobs[:]
        del self.rewards[:]
        del self.is_terminals[:]
        
    def __len__(self):
        return len(self.actions)

In [3]:
class ActorCritic(nn.Module):
    def __init__(self, state_dim, action_dim, n_latent_var):
        super(ActorCritic, self).__init__()

        # actor
        self.action_layer = nn.Sequential(
                nn.Linear(state_dim, n_latent_var),
                nn.Tanh(),
                nn.Linear(n_latent_var, n_latent_var),
                nn.Tanh(),
                nn.Linear(n_latent_var, action_dim),
                nn.Softmax(dim=-1)
                )
        
        # critic
        self.value_layer = nn.Sequential(
                nn.Linear(state_dim, n_latent_var),
                nn.Tanh(),
                nn.Linear(n_latent_var, n_latent_var),
                nn.Tanh(),
                nn.Linear(n_latent_var, 1)
                )
        
    def forward(self):
        raise NotImplementedError
        
    def act(self, state, memory):
        state = torch.from_numpy(state).float().to(device) 
        action_probs = self.action_layer(state)
        dist = Categorical(action_probs)
        action = dist.sample()
        
        memory.states.append(state)
        memory.actions.append(action)
        memory.logprobs.append(dist.log_prob(action))
        
        return action.item()
    
    def evaluate(self, state, action):
        action_probs = self.action_layer(state)
        dist = Categorical(action_probs)
        
        action_logprobs = dist.log_prob(action)
        dist_entropy = dist.entropy()
        
        state_value = self.value_layer(state)
        
        return action_logprobs, torch.squeeze(state_value), dist_entropy

In [7]:
class PPO:
    def __init__(self, state_dim, action_dim, n_latent_var, lr, betas, gamma, lambd, K_epochs, eps_clip):
        self.lr = lr
        self.betas = betas
        self.gamma = gamma
        self.lambd = lambd
        self.eps_clip = eps_clip
        self.K_epochs = K_epochs
        
        self.policy = ActorCritic(state_dim, action_dim, n_latent_var).to(device)
        self.optimizer = torch.optim.Adam(self.policy.parameters(), lr=lr, betas=betas)
        self.policy_old = ActorCritic(state_dim, action_dim, n_latent_var).to(device)
        self.policy_old.load_state_dict(self.policy.state_dict())
        
        self.MseLoss = nn.MSELoss()
        
        
    def _returns_advantages(self, state_values, memory, method="GAE", normalize=False):
        
        assert method in ["MC", "GAE", "nTD"]
        
        if method=="MC": 
            
            # Monte Carlo estimate of state rewards:
            rewards = []
            discounted_reward = 0
            for reward, is_terminal in zip(reversed(memory.rewards), reversed(memory.is_terminals)):
                if is_terminal:
                    discounted_reward = 0
                discounted_reward = reward + (self.gamma * discounted_reward)
                rewards.insert(0, discounted_reward)

            # Normalizing the rewards:
            rewards = torch.tensor(rewards).to(device)
            if normalize:
                rewards = (rewards - rewards.mean()) / (rewards.std() + 1e-5)
                
            advantages = rewards - state_values.detach()
            
            return rewards, advantages
        
        
        elif method=="GAE":
            
            rewards = np.array(memory.rewards)
            if normalize:
                rewards = (rewards - rewards.mean()) / (rewards.std() + 1e-5)
    
            returns = []
            gae = 0 

            for i in reversed(range(len(memory)-1)):
                next_value = state_values[i+1]
                delta = rewards[i] + self.gamma*next_value*(1-memory.is_terminals[i]) - state_values[i]
                gae = delta + self.gamma*self.lambd*(1-memory.is_terminals[i])*gae
                returns.insert(0,gae + state_values[i])

            returns = torch.FloatTensor(returns).to(device)
            advantages = returns - torch.FloatTensor(state_values[:-1]).to(device)
            
            return returns, advantages 
                

    
    def update(self, memory):   
        # Monte Carlo estimate of state rewards:
        rewards = []
        discounted_reward = 0
        for reward, is_terminal in zip(reversed(memory.rewards), reversed(memory.is_terminals)):
            if is_terminal:
                discounted_reward = 0
            discounted_reward = reward + (self.gamma * discounted_reward)
            rewards.insert(0, discounted_reward)
        
        # Normalizing the rewards:
        rewards = torch.tensor(rewards).to(device)
        rewards = (rewards - rewards.mean()) / (rewards.std() + 1e-5)
        
        # convert list to tensor
        old_states = torch.stack(memory.states).to(device).detach()
        old_actions = torch.stack(memory.actions).to(device).detach()
        old_logprobs = torch.stack(memory.logprobs).to(device).detach()
        
        # Optimize policy for K epochs:
        for _ in range(self.K_epochs):
            # Evaluating old actions and values :
            logprobs, state_values, dist_entropy = self.policy.evaluate(old_states, old_actions)
            
            # Finding the ratio (pi_theta / pi_theta__old):
            ratios = torch.exp(logprobs - old_logprobs.detach())
                
            # Finding Surrogate Loss:
            advantages = rewards - state_values.detach()
            surr1 = ratios * advantages
            surr2 = torch.clamp(ratios, 1-self.eps_clip, 1+self.eps_clip) * advantages
            loss = -torch.min(surr1, surr2) + 0.5*self.MseLoss(state_values, rewards) - 0.01*dist_entropy
            
            # take gradient step
            self.optimizer.zero_grad()
            loss.mean().backward()
            self.optimizer.step()
        
        # Copy new weights into old policy:
        self.policy_old.load_state_dict(self.policy.state_dict())

In [8]:
def train(env_name):
    ############## Hyperparameters ##############
    # creating environment
    env = gym.make(env_name)
    state_dim = env.observation_space.shape[0]
    action_dim = 4
    render = False
    solved_reward = 230         # stop training if avg_reward > solved_reward
    log_interval = 20           # print avg reward in the interval
    max_episodes = 50000        # max training episodes
    max_timesteps = 300         # max timesteps in one episode
    n_latent_var = 64           # number of variables in hidden layer
    update_timestep = 2000      # update policy every n timesteps
    lr = 0.002
    betas = (0.9, 0.999)
    gamma = 0.99                # discount factor   
    lambd = 0.95                # GAE factor
    K_epochs = 4                # update policy for K epochs
    eps_clip = 0.2              # clip parameter for PPO
    random_seed = None
    #############################################
    
    if random_seed:
        torch.manual_seed(random_seed)
        env.seed(random_seed)
    
    memory = Memory()
    ppo = PPO(state_dim, action_dim, n_latent_var, lr, betas, gamma, lambd, K_epochs, eps_clip)
    print(lr,betas)
    
    # logging variables
    running_reward = 0
    avg_length = 0
    timestep = 0
    
    # training loop
    for i_episode in range(1, max_episodes+1):
        state = env.reset()
        for t in range(max_timesteps):
            timestep += 1
            
            # Running policy_old:
            action = ppo.policy_old.act(state, memory)
            state, reward, done, _ = env.step(action)
            
            # Saving reward and is_terminal:
            memory.rewards.append(reward)
            memory.is_terminals.append(done)
            
            # update if its time
            if timestep % update_timestep == 0:
                ppo.update(memory)
                memory.clear_memory()
                timestep = 0
            
            running_reward += reward
            if render:
                env.render()
            if done:
                break
                
        avg_length += t
        
        # stop training if avg_reward > solved_reward
        if running_reward > (log_interval*solved_reward):
            print("########## Solved! ##########")
            torch.save(ppo.policy.state_dict(), './PPO_{}.pth'.format(env_name))
            break
            
        # logging
        if i_episode % log_interval == 0:
            avg_length = int(avg_length/log_interval)
            running_reward = int((running_reward/log_interval))
            
            print('Episode {} \t avg length: {} \t reward: {}'.format(i_episode, avg_length, running_reward))
            running_reward = 0
            avg_length = 0

In [9]:
# standard implementation
train("LunarLander-v2")

0.002 (0.9, 0.999)
Episode 20 	 avg length: 83 	 reward: -122
Episode 40 	 avg length: 86 	 reward: -156
Episode 60 	 avg length: 84 	 reward: -160
Episode 80 	 avg length: 83 	 reward: -119
Episode 100 	 avg length: 83 	 reward: -121
Episode 120 	 avg length: 81 	 reward: -106
Episode 140 	 avg length: 80 	 reward: -120
Episode 160 	 avg length: 86 	 reward: -104
Episode 180 	 avg length: 99 	 reward: -128
Episode 200 	 avg length: 104 	 reward: -99
Episode 220 	 avg length: 109 	 reward: -89
Episode 240 	 avg length: 140 	 reward: -62
Episode 260 	 avg length: 156 	 reward: -65
Episode 280 	 avg length: 209 	 reward: -15
Episode 300 	 avg length: 237 	 reward: -3
Episode 320 	 avg length: 221 	 reward: -10
Episode 340 	 avg length: 275 	 reward: 9
Episode 360 	 avg length: 285 	 reward: 9
Episode 380 	 avg length: 231 	 reward: 35
Episode 400 	 avg length: 198 	 reward: 48
Episode 420 	 avg length: 274 	 reward: 80
Episode 440 	 avg length: 244 	 reward: 62
Episode 460 	 avg length: 

KeyboardInterrupt: 

In [ ]:
# + GAE
train("LunarLander-v2")

In [10]:
# standard implementation (no normalisation)
train('CartPole-v1')

0.002 (0.9, 0.999)


AssertionError: 3 (<class 'int'>) invalid